In [ ]:
import requests
import unicodedata
import json
import pandas as pd
import time
import random

def turnPages(url, state):
    cars = []
    page = 1
    webmotors = getCars(url, page)
    
    while len(webmotors['SearchResults']) == 24:
        cars.extend(treatCars(webmotors['SearchResults']))
        page += 1
        time.sleep(random.randint(2, 4))
        print(len(cars), page)
        webmotors = getCars(url, page)
    
    if len(webmotors['SearchResults']) != 0:
        cars.extend(treatCars(webmotors['SearchResults']))
        print(len(cars), page)
    
    doCSV(cars, state)

def getCars(url, page):
    full_url = f'{url}{page}'
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
    response = requests.get(full_url, headers=headers)
    return json.loads(response.content)

def treatCars(carList):
    treatedCars = []
    for car in carList:
        try:
            fipe = car['FipePercent']
        except:
            fipe = 666

        make = unicodedata.normalize("NFD", car['Specification']['Make']['Value']).encode("ascii", "ignore").decode("utf-8")
        model = unicodedata.normalize("NFD", car['Specification']['Model']['Value']).encode("ascii", "ignore").decode("utf-8")
        version = unicodedata.normalize("NFD", car['Specification']['Version']['Value']).encode("ascii", "ignore").decode("utf-8")
        ports = car['Specification']['NumberPorts']
        anoFabricacao = int(car['Specification']['YearFabrication'])
        anoModelo = int(car['Specification']['YearModel'])
        Id = car['UniqueId']
        link = f'https://www.webmotors.com.br/comprar/{make}/{model}/{version.replace(" ", "-")}/{ports}-portas/{anoFabricacao}-{anoModelo}/{Id}'
        
        treatedCar = {
            'Ano Fabricacao': int(anoFabricacao),
            'Ano Modelo': int(anoModelo),
            'Cidade': car['Seller']['City'],
            'Cor': car['Specification']['Color']['Primary'],
            'Fipe': fipe,
            'Id': Id,
            'KM': int(car['Specification']['Odometer']),
            'Marca': make,
            'Modelo': model,
            'Tipo Vendedor': car['Seller']['SellerType'],
            'Transmissao': car['Specification']['Transmission'],
            'Valor': int(car['Prices']['Price']),
            'Versao': version,
            'Link': link
        }
        treatedCars.append(treatedCar)
    
    return treatedCars

def doCSV(cars, state):
    df = pd.DataFrame(data=cars)
    column_names = ['Id', 'Fipe', 'Marca', 'Modelo', 'Valor', 'Ano Fabricacao', 'Ano Modelo', 'KM', 'Transmissao', 'Cor', 'Versao', 'Cidade', 'Tipo Vendedor', 'Link']
    mydf = df.reindex(columns=column_names)
    
    save_path = f'WebMotors - {state}.csv'
    mydf.to_csv(save_path, encoding='utf-8', index=False)

# Lista de URLs e estados
states_urls = {
    rj = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Frj%3Festadocidade%3DRio%2520de%2520Janeiro%26tipoveiculo%3Dcarros-usados&actualPage=',
    sp = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Fsp%3Festadocidade%3DS%25C3%25A3o%2520Paulo%26tipoveiculo%3Dcarros-usados&actualPage=',
    mg = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Fmg%3Festadocidade%3DMinas%2520Gerais%26tipoveiculo%3Dcarros-usados&actualPage=',
    es = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Fes%3Festadocidade%3DEsp%25C3%25ADrito%2520Santo%26tipoveiculo%3Dcarros-usados&actualPage=',
    rs = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Frs%3Festadocidade%3DRio%2520Grande%2520do%2520Sul%26tipoveiculo%3Dcarros-usados&actualPage=',
    sc = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Fsc%3Festadocidade%3DSanta%2520Catarina%26tipoveiculo%3Dcarros-usados&actualPage=',
    pr = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Fpr%3Festadocidade%3DParan%25C3%25A1%26tipoveiculo%3Dcarros-usados&actualPage='
    ac = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Fac%3Festadocidade%3DAcre%26tipoveiculo%3Dcarros-usados&actualPage=',
    am = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Fam%3Festadocidade%3DAmazonas%26tipoveiculo%3Dcarros-usados&actualPage=',
    ap = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Fap%3Festadocidade%3DAmapa%26tipoveiculo%3Dcarros-usados&actualPage=',
    pa = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Fpa%3Festadocidade%3DPar%25C3%25A1%26tipoveiculo%3Dcarros-usados&actualPage=',
    ro = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Fro%3Festadocidade%3DRond%25C3%25B4nia%26tipoveiculo%3Dcarros-usados&actualPage=',
    rr = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Frr%3Festadocidade%3DRoraima%26tipoveiculo%3Dcarros-usados&actualPage=',
    to = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Fto%3Festadocidade%3DTocantins%26tipoveiculo%3Dcarros-usados&actualPage=',
    df = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Fdf%3Festadocidade%3DDistrito%2520Federal%26tipoveiculo%3Dcarros-usados&actualPage=',
    go = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Fgo%3Festadocidade%3DGoi%25C3%25A1s%26tipoveiculo%3Dcarros-usados&actualPage=',
    ms = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Fms%3Festadocidade%3DMato%2520Grosso%2520do%2520Sul%26tipoveiculo%3Dcarros-usados&actualPage=',
    mt = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Fmt%3Festadocidade%3DMato%2520Grosso%26tipoveiculo%3Dcarros-usados&actualPage=',
    al = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Fal%3Festadocidade%3DAlagoas%26tipoveiculo%3Dcarros-usados&actualPage=',
    ba = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Fba%3Festadocidade%3DBahia%26tipoveiculo%3Dcarros-usados&actualPage=',
    ce = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Fce%3Festadocidade%3DCear%25C3%25A1%26tipoveiculo%3Dcarros-usados&actualPage=',
    ma = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Fma%3Festadocidade%3DMaranh%25C3%25A3o%26tipoveiculo%3Dcarros-usados&actualPage=',
    pb = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Fpb%3Festadocidade%3DPara%25C3%25ADba%26tipoveiculo%3Dcarros-usados&actualPage=',
    pe = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Fpe%3Festadocidade%3DPernambuco%26tipoveiculo%3Dcarros-usados&actualPage=',
    pi = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Fpi%3Festadocidade%3DPiau%25C3%25AD%26tipoveiculo%3Dcarros-usados&actualPage=',
    rn = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Frn%3Festadocidade%3DRio%2520Grande%2520do%2520Norte%26tipoveiculo%3Dcarros-usados&actualPage=',
    se = 'https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Fse%3Festadocidade%3DSergipe%26tipoveiculo%3Dcarros-usados&actualPage='
}

# Executa o procedimento para cada estado
for state, url in states_urls.items():
    turnPages(url, state)
